In [5]:
import pandas as pd
import numpy as np
# import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/all_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels2021.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels2020.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno
docnos21 = qrels21.docno
docnos20 = qrels20.docno

In [3]:
import os
# rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

objective="ndcg_cut_5"
for i, (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"dirichletlm.mu" : 2500})
            tuned_dlm, parameters = pt.KFoldGridSearch(
                dlm,
                {dlm: {"dirichletlm.mu" : list(range(0,10000,250))}},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, 
                                names=['tuned_dlm_unjudged'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos20)]], topics20, qrels, 
                                names=['tuned_dlm_unjudged20'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos21)]], topics21, qrels, 
                                names=['tuned_dlm_unjudged21'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                                    

            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
Fold 1
Best ndcg_cut_5 is 0.652732
Best setting is ['BR(DirichletLM) dirichletlm.mu=3000']
Fold 2
Best ndcg_cut_5 is 0.513757
Best setting is ['BR(DirichletLM) dirichletlm.mu=750']
\begin{tabular}{llrrr}
\toprule
{} &                name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  tuned\_dlm\_unjudged &    0.697009 &      0.64147 &  0.594445 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                  name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  tuned\_dlm\_unjudged20 &    0.766289 &     0.679857 &  0.573064 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                  name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  tuned\_dlm\_unjudged21 &     0.66512 &     0.660926 &  0.615399 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2391695/3058016727.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels,
/tmp/ipykernel_2391695/3058016727.py:24: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos20)]], topics20, qrels,
/tmp/ipykernel_2391695/3058016727.py:27: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementa

In [4]:
'''
Example of simple tuning dlm and its not saving to the folder 
'''

import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
metrics="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"dirichletlm.mu" : 2500})
            tuned_dlm, parameters = pt.KFoldGridSearch(
                dlm,
                {dlm: {"dirichletlm.mu" : list(range(0,10000,250))}},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_unjudged'],
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_unjudged20'],
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] + '_tuned_dlm_unjudged21'],
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            

            i += 1
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.607973
Best setting is ['BR(DirichletLM) dirichletlm.mu=3750']
Fold 2
Best ndcg_cut_5 is 0.450963
Best setting is ['BR(DirichletLM) dirichletlm.mu=5750']


/tmp/ipykernel_2391695/2422250554.py:25: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_unjudged'],
/tmp/ipykernel_2391695/2422250554.py:27: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_unjudged20'],
/tmp/ipykernel_2391695/2422250554.py:29: FutureW

\begin{tabular}{llrrr}
\toprule
{} &                            name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_tuned\_dlm\_unjudged &    0.724873 &      0.69907 &  0.700047 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                              name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &    bpref \\
\midrule
0 &  conclusions\_tuned\_dlm\_unjudged20 &    0.780337 &     0.743316 &  0.68032 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                              name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &    bpref \\
\midrule
0 &  conclusions\_tuned\_dlm\_unjudged21 &    0.714678 &     0.712991 &  0.71938 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pool_topicspd_index
Fold 1
Best ndcg_cut_5 is 0.624379
Best setting is ['BR(DirichletLM) dirichletlm.mu=2250']
Fold 2
Best ndcg_cut_5 is 0.412219
Best setting is ['BR(DirichletLM) dirichletlm.mu=

/tmp/ipykernel_2391695/2422250554.py:25: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_unjudged'],
/tmp/ipykernel_2391695/2422250554.py:27: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_unjudged20'],
/tmp/ipykernel_2391695/2422250554.py:29: FutureW

\begin{tabular}{llrrr}
\toprule
{} &                            name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_tuned\_dlm\_unjudged &    0.712392 &     0.695125 &  0.693835 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                              name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_tuned\_dlm\_unjudged20 &    0.770778 &     0.736075 &  0.676231 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                              name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_tuned\_dlm\_unjudged21 &    0.704461 &     0.713294 &  0.711088 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/topicspd_index
Fold 1
Best ndcg_cut_5 is 0.605028
Best setting is ['BR(DirichletLM) dirichletlm.mu=9000']
Fold 2
Best ndcg_cut_5 is 0.454940
Best setting is ['BR(DirichletLM) dirichletlm.m

...

/tmp/ipykernel_2391695/2422250554.py:25: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_unjudged'],
/tmp/ipykernel_2391695/2422250554.py:27: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_unjudged20'],
/tmp/ipykernel_2391695/2422250554.py:29: FutureW

\begin{tabular}{llrrr}
\toprule
{} &                                        name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_conclusions\_tuned\_dlm\_unjudged &    0.732391 &     0.705404 &  0.699466 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                          name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  pool\_topics\_conclusions\_tuned\_dlm\_unjudged20 &    0.780413 &     0.742215 &  0.675033 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                          name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &    bpref \\
\midrule
0 &  pool\_topics\_conclusions\_tuned\_dlm\_unjudged21 &    0.724327 &     0.725282 &  0.72341 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusions_leakage/pd_index
Fold 1
Best ndcg_cut_5 is 0.623032
Best setting is ['BR(DirichletLM) dirichletlm.mu=60

/tmp/ipykernel_2391695/2422250554.py:25: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_unjudged'],
/tmp/ipykernel_2391695/2422250554.py:27: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_unjudged20'],
/tmp/ipykernel_2391695/2422250554.py:29: FutureW

\begin{tabular}{llrrr}
\toprule
{} &                                    name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_leakage\_tuned\_dlm\_unjudged &    0.737078 &     0.707679 &  0.695927 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                      name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_leakage\_tuned\_dlm\_unjudged20 &     0.79072 &     0.742226 &  0.671135 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                      name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  conclusions\_leakage\_tuned\_dlm\_unjudged21 &    0.722354 &      0.72893 &  0.720224 \\
\bottomrule
\end{tabular}


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pools_conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.618876
Best setting is ['BR(DirichletLM) dirichletlm.mu=2750']
Fold 2
Best ndcg_cut_5 

/tmp/ipykernel_2391695/2422250554.py:25: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_unjudged'],
/tmp/ipykernel_2391695/2422250554.py:27: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_unjudged20'],
/tmp/ipykernel_2391695/2422250554.py:29: FutureW

In [6]:
'''
Example of simple tuning dlm and its not saving to the folder 
'''

import os
rootdir = "/workspace/src/grid_search_right_qrels/super_folder/"
folders = ['marco-pretrained']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"dirichletlm.mu" : 2500})
            tuned_dlm, parameters = pt.KFoldGridSearch(
                dlm,
                {dlm: {"dirichletlm.mu" : list(range(0,10000,250))}},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_unjudged'],
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_unjudged20'],
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] + '_tuned_dlm_unjudged21'],
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            

            i += 1
            


/workspace/src/grid_search_right_qrels/super_folder/
Fold 1
Best ndcg_cut_5 is 0.455074
Best setting is ['BR(DirichletLM) dirichletlm.mu=1000']
Fold 2
Best ndcg_cut_5 is 0.250555
Best setting is ['BR(DirichletLM) dirichletlm.mu=1500']
\begin{tabular}{llrrr}
\toprule
{} &                                 name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_dlm\_unjudged &     0.59213 &     0.608695 &  0.610536 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                   name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_dlm\_unjudged20 &    0.606138 &     0.625352 &  0.601926 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                   name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_dlm\_unjudged21 &    0.601868 &     0.638206 &  0.618973 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2391695/2876488838.py:25: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_unjudged'],
/tmp/ipykernel_2391695/2876488838.py:27: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_unjudged20'],
/tmp/ipykernel_2391695/2876488838.py:29: FutureW